In [2]:
import os
import pandas as pd
import numpy as np
import time
import cv2
import tensorflow as tf
import keras.backend as K
import keras

In [1]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15455932450995157296
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3665166336
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11741565625213436988
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
]


In [3]:
data_path="dataset/"
label=pd.read_csv(data_path+"label.csv")
file_names=list(label['file_name'])
labels=list(label['label'])

In [4]:
label.label.unique()

array(['meningioma_tumor', 'no_tumor', 'glioma_tumor', 'pituitary_tumor'],
      dtype=object)

In [5]:
len(label.label.unique())

4

In [6]:
label

,file_name,label
0,IMAGE_0000.jpg,meningioma_tumor
1,IMAGE_0001.jpg,no_tumor
2,IMAGE_0002.jpg,meningioma_tumor
3,IMAGE_0003.jpg,glioma_tumor
4,IMAGE_0004.jpg,meningioma_tumor
...,...,...
2995,IMAGE_2995.jpg,no_tumor
2996,IMAGE_2996.jpg,meningioma_tumor
2997,IMAGE_2997.jpg,glioma_tumor
2998,IMAGE_2998.jpg,glioma_tumor


In [12]:
label['task_a_label'] = label['label'].apply(lambda x: 0 if x == "no_tumor" else 1)

In [16]:
label = label.drop(columns = ['label'])

In [17]:
label

,file_name,task_a_label
0,IMAGE_0000.jpg,1
1,IMAGE_0001.jpg,0
2,IMAGE_0002.jpg,1
3,IMAGE_0003.jpg,1
4,IMAGE_0004.jpg,1
...,...,...
2995,IMAGE_2995.jpg,0
2996,IMAGE_2996.jpg,1
2997,IMAGE_2997.jpg,1
2998,IMAGE_2998.jpg,1
